In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

import numpy as np
import os
from PIL import Image
from collections import OrderedDict
from generator import Generator

In [3]:
torch_g = Generator(dim=28, channels=1)

In [4]:
z = np.ones((1, 100)) * 0.5

In [5]:
raw_data = np.load("datasets/omniglot_data.npy")

In [6]:
inp = np.array([raw_data[1200][0]], dtype="float64")

In [7]:
torch_g = Generator(dim=28, channels=1)
torch_g.eval()

Generator(
  (encode0): Sequential(
    (pad): _SamePad()
    (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2))
    (relu): LeakyReLU(negative_slope=0.2)
    (batchnorm): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (encode1): _EncoderBlock(
    (pre_conv): Sequential(
      (pad): _SamePad()
      (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
    )
    (conv0): Sequential(
      (pad): _SamePad()
      (conv): Conv2d(65, 64, kernel_size=(3, 3), stride=(1, 1))
      (relu): LeakyReLU(negative_slope=0.2)
      (batchnorm): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    )
    (conv1): Sequential(
      (pad): _SamePad()
      (conv): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
      (relu): LeakyReLU(negative_slope=0.2)
      (batchnorm): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    )
    (conv2): Sequential(
      (pad): _SamePad()
      (conv):

In [10]:
input_images = torch.ones((32, 1, 28, 28))
z = torch.randn((32, 100))

In [31]:
torch_g.sample(input_images).shape

torch.Size([32, 1, 28, 28])

In [ ]:
torch_inp = torch.tensor(inp).transpose(1, 3).transpose(2, 3).float()
torch_out = torch_g(torch_inp, torch.tensor(z).float())
torch_out[0][0][0]

In [ ]:
def render_image_arr(arr):
    arr = np.uint8(arr * 256)
    arr = arr.reshape(arr.shape[:-1])
    display(Image.fromarray(arr, mode='L').resize((224, 224)))
    
def render_torch(arr):
    arr = np.uint8(arr * 256)
    display(Image.fromarray(arr, mode='L').resize((224, 224)))

In [ ]:
# Render torch
refined_torch_out = torch_out.detach().numpy() * 0.5 + 0.5

render_torch(refined_torch_out[0][0])

In [ ]:
from torchvision.models.densenet import DenseNet
net = DenseNet(growth_rate=16, block_config=(3,3,3,3), num_classes=10, drop_rate=0.0)
net

In [ ]:
def convert_bn(net):
    for name in net._modules.keys():
        if "norm" in name:
            net._modules[name] = nn.Identity()
        else:
            convert_bn(net._modules[name])

In [ ]:
convert_bn(net)
net

In [54]:
mod = nn.BatchNorm2d(64, affine=True)